# Creating Ground Truth Dataset

In [64]:
import pandas as pd 
import json
import ast
import re
# Fuzzy Matching with FuzzyWuzzy
from fuzzywuzzy import fuzz
from collections import defaultdict

In [65]:
# Import sample data
sample_df = pd.read_csv('../input_data/normalized_samples.csv')
sample_df

,sample_idx,id,base_id,sources,names,categories,confidence,websites,socials,emails,...,address_state_norm,address_postcode_norm,base_address_street_norm,base_address_city_norm,base_address_state_norm,base_address_postcode_norm,address_norm,base_address_norm,categories_norm,base_categories_norm
0,0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{'property': '', 'dataset': 'meta', 'record_i...","{'primary': ""Goin' Postal Jacksonville""}","{'primary': 'shipping_center', 'alternate': ['...",0.996261,['http://www.goinpostaljacksonville.com/'],['https://www.facebook.com/463273470392736'],NaN,...,FL,32256,7643 gate pkwy ste 104,jacksonville,FL,32256,"7643 gate pkwy, jacksonville, FL, 32256","7643 gate pkwy ste 104, jacksonville, FL, 32256","['shipping_center', 'freight_and_cargo_service...","['vehicle_shipping', 'courier_and_delivery_ser..."
1,1,08f29a456e42e5830324637954145c50,1125899907111860,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Valley Transmission'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,['http://valleytransca.com/'],['https://www.facebook.com/107150256087765'],NaN,...,CA,92021,1158 n 2nd st,el cajon,CA,92021,"1158 n 2nd st, el cajon, CA, 92021","1158 n 2nd st, el cajon, CA, 92021","['automotive_repair', 'automotive', 'professio...","['transmission_repair', 'automotive']"
2,2,08fbcd0030da5323031bcafa8c2fa0dc,844424934845986,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Mazda Nelspruit'},"{'primary': 'car_dealer', 'alternate': ['autom...",0.997899,['http://bit.ly/NelspruitMazda'],['https://www.facebook.com/113070635372645'],NaN,...,MP,01201,49 emnotweni avenue,mbombela,NaN,01201,"mazda nelspruit, 49 emnotweni avenue, riversid...","49 emnotweni avenue, mbombela, 01201","['car_dealer', 'automotive', 'automotive_dealer']","['used_car_dealer', 'automotive']"
3,3,1407374885933937,844424932146366,"[{'dataset': 'Microsoft', 'record_id': '140737...","{'primary': 'Red Wing - Roswell, GA'}","{'primary': 'shoe_store', 'alternate': ['onlin...",0.770000,['https://stores.redwingshoes.com/roswell-ga'],NaN,NaN,...,GA,30076,875 mansell rd ste b1,roswell,GA,30076,"875 mansell rd b1, roswell, GA, 30076","875 mansell rd ste b1, roswell, GA, 30076","['shoe_store', 'online_shop', 'retail']","['shoe_store', 'online_shop', 'retail']"
4,4,08f3956260b9e14003feca2bf0764d0c,1407374887472291,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Norauto España'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,['https://centro.norauto.es/alacant/finestrat/...,['https://www.facebook.com/295967644448975'],NaN,...,NaN,03509,calle rei jaume i 9,alicante,ALACANT,03509,"carrer del rei jaume i, 9, finestrat, 03509","calle rei jaume i 9, alicante, ALACANT, 03509","['automotive_repair', 'car_dealer', 'automotiv...","['brake_service_and_repair', 'auto_body_shop',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,08f1e1251b0a66c003a195f0bf6b4bc9,502a184a011ce8eabd5934fb,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Studio Legale Associato Petracci ...,"{'primary': 'lawyer', 'alternate': ['criminal_...",0.286713,['http://www.petraccimarin.it'],['https://www.facebook.com/121776774503515'],NaN,...,36,34125,via beccaria 3,trieste,FVG,34125,"viale 20 settembre, 16, trieste, 36, 34125","via beccaria 3, trieste, FVG, 34125","['lawyer', 'criminal_defense_law']",['business_and_professional_services_>_office']
1996,1996,08fa80b035674b6a03379ca9c1ecea9f,50364256e4b0573431043dd4,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Farmácias Pague Menos'},"{'primary': 'pharmacy', 'alternate': ['pharmac...",0.792793,['https://www.paguemenos.com.br/'],['https://www.facebook.com/2204406756483378'],NaN,...,MS,79601,"av. antônio trajano, 583",três lagoas,MS,NaN,"avenida antônio trajano, 538, três lagoas, MS,...","av. antônio trajano, 583, três lagoas, MS","['pharmacy', 'pharmaceutical_companies', 'beau...",['ret

In [66]:
# Import Yelp dataset
yelp_df = pd.read_csv('../input_data/normalized_yelp.csv')
yelp_df.head()

,business_id,name,address,city,state,postal_code,categories,address_norm,city_norm,state_norm,postal_code_norm,name_norm,categories_norm,address_string_norm
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,"Doctors, Traditional Chinese Medicine, Naturop...","1616 chapala st, ste 2",santa barbara,CA,93101,abby rappoport lac cmq,"['doctors', 'traditional_chinese_medicine', 'n...","1616 chapala st, ste 2, santa barbara, CA, 93101"
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,"Shipping Centers, Local Services, Notaries, Ma...",87 grasso plaza shopping center,affton,MO,63123,ups store,"['shipping_centers', 'local_services', 'notari...","87 grasso plaza shopping center, affton, MO, 6..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,"Department Stores, Shopping, Fashion, Home & G...",5255 e broadway blvd,tucson,AZ,85711,target,"['department_stores', 'shopping', 'fashion', '...","5255 e broadway blvd, tucson, AZ, 85711"
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",935 race st,philadelphia,PA,19107,st honore pastries,"['restaurants', 'food', 'bubble_tea', 'coffee_...","935 race st, philadelphia, PA, 19107"
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,"Brewpubs, Breweries, Food",101 walnut st,green lane,PA,18054,perkiomen valley brewery,"['brewpubs', 'breweries', 'food']","101 walnut st, green lane, PA, 18054"


In [67]:
# Parse string columns back to dictionaries and lists

def safe_eval(x):
    """Safely evaluate string representation of dict/list back to Python object"""
    if pd.isna(x) or x == '' or x == 'nan':
        return None
    if isinstance(x, (dict, list)):
        return x
    try:
        return ast.literal_eval(x)
    except:
        return x

# Categories are still lists, need parsing
sample_df['categories_norm'] = sample_df['categories_norm'].apply(safe_eval)
sample_df['base_categories_norm'] = sample_df['base_categories_norm'].apply(safe_eval)
yelp_df['categories_norm'] = yelp_df['categories_norm'].apply(safe_eval)

# Addresses are now strings, no parsing needed

## Fuzzy Matching: Sample Data vs Yelp Data

This section uses fuzzy matching to find the best Yelp business for each sample record, then evaluates which attributes (base vs non-base) are most accurate.

In [68]:
def fuzzy_name_similarity(name1, name2):
    """Calculate fuzzy similarity between two names using token sort ratio"""
    if not name1 or not name2:
        return 0.0
    return fuzz.token_sort_ratio(name1, name2) / 100.0

def fuzzy_address_similarity(addr_str1, addr_str2):
    """Calculate fuzzy similarity between two address strings"""
    if not addr_str1 or not addr_str2:
        return 0.0
    
    # Use token sort ratio for address strings (handles component order differences)
    return fuzz.token_sort_ratio(addr_str1, addr_str2) / 100.0

def jaccard_similarity(set1, set2):
    """Calculate Jaccard similarity between two sets"""
    if not set1 and not set2:
        return 1.0
    if not set1 or not set2:
        return 0.0
    
    s1 = set(set1)
    s2 = set(set2)
    
    intersection = len(s1.intersection(s2))
    union = len(s1.union(s2))
    
    return intersection / union if union > 0 else 0.0

def find_best_match(sample_row, yelp_df, max_candidates=2000, min_confidence=0.4):
    """
    Find the best matching Yelp business for a sample row using improved fuzzy matching
    
    Improvements:
    - Uses both state AND city filtering for better geographic constraints
    - Requires BOTH name and address to match reasonably well (no single-attribute matches)
    - Higher weight on address (0.5) since addresses are more unique than names
    - Minimum confidence threshold to reject poor matches
    - Increased candidate pool to 2000 for better coverage
    """
    
    # Get normalized attributes from sample row
    sample_name = sample_row['name_norm']
    sample_base_name = sample_row['base_name_norm']
    sample_address = sample_row['address_norm']  # Now a string
    sample_base_address = sample_row['base_address_norm']  # Now a string
    
    # Extract geographic info for filtering (use dedicated columns)
    sample_state = sample_row.get('address_state_norm', '')
    sample_city = sample_row.get('address_city_norm', '')
    
    # Multi-level geographic filtering for better precision
    yelp_candidates = yelp_df
    
    # First try: Filter by both state AND city
    if sample_state and sample_city:
        city_state_matches = yelp_df[
            (yelp_df['state_norm'] == sample_state) & 
            (yelp_df['city_norm'] == sample_city)
        ]
        if len(city_state_matches) >= 10:  # Use city+state if we have enough candidates
            yelp_candidates = city_state_matches.head(max_candidates)
        elif sample_state:  # Fall back to state only
            yelp_candidates = yelp_df[yelp_df['state_norm'] == sample_state].head(max_candidates)
    elif sample_state:  # Only state available
        yelp_candidates = yelp_df[yelp_df['state_norm'] == sample_state].head(max_candidates)
    else:
        # No geographic info - use first N candidates (risky)
        yelp_candidates = yelp_df.head(max_candidates)
    
    if len(yelp_candidates) == 0:
        # Fallback: no geographic matches, use broader set
        yelp_candidates = yelp_df.head(max_candidates)
    
    best_match = None
    best_score = 0.0
    
    for idx, yelp_row in yelp_candidates.iterrows():
        yelp_name = yelp_row['name_norm']
        yelp_address = yelp_row['address_string_norm']  # Now a string
        
        # Calculate name similarities (try both base and non-base)
        name_sim = fuzzy_name_similarity(sample_name, yelp_name)
        base_name_sim = fuzzy_name_similarity(sample_base_name, yelp_name)
        
        # Calculate address similarities (try both base and non-base)
        addr_sim = fuzzy_address_similarity(sample_address, yelp_address)
        base_addr_sim = fuzzy_address_similarity(sample_base_address, yelp_address)
        
        # Use the better of base/non-base for each attribute
        best_name_sim = max(name_sim, base_name_sim)
        best_addr_sim = max(addr_sim, base_addr_sim)
        
        # CRITICAL IMPROVEMENT: Require BOTH name and address to match reasonably
        # Reject matches where only one attribute is high (likely false positive)
        if best_name_sim < 0.3 or best_addr_sim < 0.2:
            continue  # Skip this candidate - not enough evidence
        
        # Combined score with higher weight on address (addresses are more unique)
        combined_score = (best_name_sim * 0.5) + (best_addr_sim * 0.5)
        
        if combined_score > best_score:
            best_score = combined_score
            best_match = {
                'yelp_idx': idx,
                'yelp_business_id': yelp_row['business_id'],
                'yelp_name': yelp_name,
                'yelp_address': yelp_address,
                'yelp_categories': yelp_row.get('categories_norm', []),
                'name_similarity': name_sim,
                'base_name_similarity': base_name_sim,
                'address_similarity': addr_sim,
                'base_address_similarity': base_addr_sim,
                'combined_score': combined_score
            }
    
    # Apply minimum confidence threshold - reject poor matches
    if best_match and best_match['combined_score'] < min_confidence:
        return None
    
    return best_match

# Test on one sample

if len(sample_df) > 0:
    test_row = sample_df.iloc[0]
    print(f"Testing on first sample row...")
    print(f"Sample name: '{test_row['name_norm']}'")
    print(f"Sample address: {test_row['address_norm']}")
    
    test_match = find_best_match(test_row, yelp_df)
    if test_match:
        print(f"\nBest match found:")
        print(f"  Yelp name: '{test_match['yelp_name']}'")
        print(f"  Combined score: {test_match['combined_score']:.3f}")
        print(f"  Name similarity: {test_match['name_similarity']:.3f}")
        print(f"  Address similarity: {test_match['address_similarity']:.3f}")
    else:
        print("No match found!")
    

Testing on first sample row...
Sample name: 'goin' postal jacksonville'
Sample address: 7643 gate pkwy, jacksonville, FL, 32256

Best match found:
  Yelp name: 'pet trailer mobile grooming'
  Combined score: 0.470
  Name similarity: 0.550
  Address similarity: 0.320

Best match found:
  Yelp name: 'pet trailer mobile grooming'
  Combined score: 0.470
  Name similarity: 0.550
  Address similarity: 0.320


In [69]:
# Generate Ground Truth Labels with Fuzzy Matching

def generate_ground_truth():
    """Generate ground truth labels for all sample rows"""
    
    print("=== GENERATING GROUND TRUTH WITH FUZZY MATCHING ===")
    print(f"Processing {len(sample_df)} sample records...")
    
    results = []
    skipped_records = []
    
    for idx, sample_row in sample_df.iterrows():
        # Find best Yelp match
        best_match = find_best_match(sample_row, yelp_df)
        
        if not best_match:
            skipped_records.append({
                'sample_idx': sample_row['sample_idx'],
                'sample_name': sample_row['name_norm'],
                'sample_city': sample_row.get('address_city_norm', ''),
                'sample_state': sample_row.get('address_state_norm', ''),
                'reason': 'no_match_above_threshold'
            })
            continue
        
        # Get attributes for comparison
        sample_name = sample_row['name_norm']
        sample_base_name = sample_row['base_name_norm']
        sample_cats = sample_row.get('categories_norm', [])
        sample_base_cats = sample_row.get('base_categories_norm', [])
        sample_addr = sample_row['address_norm']
        sample_base_addr = sample_row['base_address_norm']
        
        yelp_name = best_match['yelp_name']
        yelp_cats = best_match['yelp_categories']
        yelp_addr = best_match['yelp_address']
        
        # Determine best attributes
        name_sim = best_match['name_similarity']
        base_name_sim = best_match['base_name_similarity']
        
        cat_sim = jaccard_similarity(sample_cats, yelp_cats)
        base_cat_sim = jaccard_similarity(sample_base_cats, yelp_cats)
        
        addr_sim = best_match['address_similarity']
        base_addr_sim = best_match['base_address_similarity']
        
        # Determine winners
        best_name = 0 if name_sim > base_name_sim else (1 if base_name_sim > name_sim else 2)
        best_categories = 0 if cat_sim > base_cat_sim else (1 if base_cat_sim > cat_sim else 2)
        best_address = 0 if addr_sim > base_addr_sim else (1 if base_addr_sim > addr_sim else 2)
        
        # Store results - use sample_idx from the CSV instead of DataFrame index
        result = {
            'sample_idx': sample_row['sample_idx'],
            'sample_id': sample_row.get('id', idx),
            'yelp_idx': best_match['yelp_idx'],
            'yelp_business_id': best_match['yelp_business_id'],
            'match_confidence': best_match['combined_score'],
            
            # Name comparison
            'best_name_source': best_name,
            'name_similarity': name_sim,
            'base_name_similarity': base_name_sim,
            'sample_name': sample_name,
            'sample_base_name': sample_base_name,
            'yelp_name': yelp_name,
            
            # Categories comparison  
            'best_categories_source': best_categories,
            'categories_similarity': cat_sim,
            'base_categories_similarity': base_cat_sim,
            'sample_categories': sample_cats,
            'sample_base_categories': sample_base_cats,
            'yelp_categories': yelp_cats,
            
            # Address comparison
            'best_address_source': best_address,
            'address_similarity': addr_sim,
            'base_address_similarity': base_addr_sim,
            'sample_address': sample_addr,
            'sample_base_address': sample_base_addr,
            'yelp_address': yelp_addr
        }
        
        results.append(result)
    
    # Print summary of skipped records
    print(f"\n{'='*60}")
    print(f"MATCHING SUMMARY:")
    print(f"{'='*60}")
    print(f"Total sample records:  {len(sample_df)}")
    print(f"Successfully matched:  {len(results)}")
    print(f"Skipped (no match):    {len(skipped_records)}")
    print(f"Match rate:            {len(results)/len(sample_df)*100:.1f}%")
    
    if skipped_records:
        print(f"\nSkipped records preview (first 10):")
        for i, skip in enumerate(skipped_records[:10]):
            print(f"  {i+1}. sample_idx={skip['sample_idx']}, name='{skip['sample_name']}', city={skip['sample_city']}, state={skip['sample_state']}")
        
        # Export skipped records for analysis
        skipped_df = pd.DataFrame(skipped_records)
        skipped_df.to_csv('../output_data/skipped_ground_truth_records.csv', index=False)
        print(f"\n✓ Exported {len(skipped_records)} skipped records to skipped_ground_truth_records.csv")
    
    return results

# Generate the ground truth labels
ground_truth_results = generate_ground_truth()

# Convert to DataFrame for easier analysis
ground_truth_df = pd.DataFrame(ground_truth_results)

=== GENERATING GROUND TRUTH WITH FUZZY MATCHING ===
Processing 2000 sample records...

MATCHING SUMMARY:
Total sample records:  2000
Successfully matched:  1639
Skipped (no match):    361
Match rate:            82.0%

Skipped records preview (first 10):
  1. sample_idx=8, name='nextcare urgent care gastonia', city=gastonia, state=NC
  2. sample_idx=20, name='audio network', city=london, state=nan
  3. sample_idx=30, name='heartland mortgage - bobbie jo haggard', city=walla walla, state=WA
  4. sample_idx=33, name='women's health boutique', city=plano, state=TX
  5. sample_idx=41, name='signwriter', city=kalamazoo, state=MI
  6. sample_idx=45, name='viking sewing gallery', city=spring, state=TX
  7. sample_idx=51, name='solar guard utah', city=west valley city, state=UT
  8. sample_idx=62, name='kincaid at legacy', city=plano, state=TX
  9. sample_idx=72, name='service king collision repair south ft worth', city=fort worth, state=TX
  10. sample_idx=73, name='spenga prosper', city=prosp

## Key Improvements to Fuzzy Matching

**Matching Strategy Changes:**
1. **Categories Omitted** - Only use names and addresses for Yelp matching (categories not reliable for matching)
2. **Geographic Constraints** - Filter by city+state first, then state-only if needed
3. **Dual-Attribute Requirement** - Both name AND address must match reasonably (minimum 0.3 name, 0.2 address)
4. **Equal Weighting** - Name and address weighted equally at 0.5 each (addresses are more unique)
5. **Minimum Confidence** - Reject matches below 0.4 combined score
6. **Larger Candidate Pool** - Search up to 2000 candidates instead of 1000
7. **Skip Tracking** - Track and export records that fail to match for analysis

In [70]:
# Analyze Ground Truth Coverage
print("=== GROUND TRUTH COVERAGE ANALYSIS ===")

if len(ground_truth_df) > 0:
    total_records = len(ground_truth_df)
    print(f"Total matched pairs: {total_records}\n")
    
    # Analyze coverage for core attributes
    print("NON-NULL COVERAGE ACROSS CORE ATTRIBUTES:")
    print("=" * 60)
    
    # Names coverage
    print("\nNAMES:")
    name_coverage = (ground_truth_df['sample_name'].notna() & (ground_truth_df['sample_name'] != '')).sum()
    base_name_coverage = (ground_truth_df['sample_base_name'].notna() & (ground_truth_df['sample_base_name'] != '')).sum()
    yelp_name_coverage = (ground_truth_df['yelp_name'].notna() & (ground_truth_df['yelp_name'] != '')).sum()
    
    print(f"  Sample names:      {name_coverage:4d} / {total_records} ({name_coverage/total_records*100:5.1f}%)")
    print(f"  Base names:        {base_name_coverage:4d} / {total_records} ({base_name_coverage/total_records*100:5.1f}%)")
    print(f"  Yelp names:        {yelp_name_coverage:4d} / {total_records} ({yelp_name_coverage/total_records*100:5.1f}%)")
    
    # Addresses coverage
    print("\nADDRESSES:")
    addr_coverage = (ground_truth_df['sample_address'].notna() & (ground_truth_df['sample_address'] != '')).sum()
    base_addr_coverage = (ground_truth_df['sample_base_address'].notna() & (ground_truth_df['sample_base_address'] != '')).sum()
    yelp_addr_coverage = (ground_truth_df['yelp_address'].notna() & (ground_truth_df['yelp_address'] != '')).sum()
    
    print(f"  Sample addresses:  {addr_coverage:4d} / {total_records} ({addr_coverage/total_records*100:5.1f}%)")
    print(f"  Base addresses:    {base_addr_coverage:4d} / {total_records} ({base_addr_coverage/total_records*100:5.1f}%)")
    print(f"  Yelp addresses:    {yelp_addr_coverage:4d} / {total_records} ({yelp_addr_coverage/total_records*100:5.1f}%)")
    
    # Categories coverage
    print("\nCATEGORIES:")
    cat_coverage = ground_truth_df['sample_categories'].apply(lambda x: x is not None and isinstance(x, list) and len(x) > 0).sum()
    base_cat_coverage = ground_truth_df['sample_base_categories'].apply(lambda x: x is not None and isinstance(x, list) and len(x) > 0).sum()
    yelp_cat_coverage = ground_truth_df['yelp_categories'].apply(lambda x: x is not None and isinstance(x, list) and len(x) > 0).sum()
    
    print(f"  Sample categories: {cat_coverage:4d} / {total_records} ({cat_coverage/total_records*100:5.1f}%)")
    print(f"  Base categories:   {base_cat_coverage:4d} / {total_records} ({base_cat_coverage/total_records*100:5.1f}%)")
    print(f"  Yelp categories:   {yelp_cat_coverage:4d} / {total_records} ({yelp_cat_coverage/total_records*100:5.1f}%)")
    
    # Summary statistics
    print("\n" + "=" * 60)
    print("MATCH QUALITY STATISTICS:")
    print("=" * 60)
    print(f"Average match confidence: {ground_truth_df['match_confidence'].mean():.3f}")
    print(f"Match confidence std:     {ground_truth_df['match_confidence'].std():.3f}")
    print(f"Min confidence:           {ground_truth_df['match_confidence'].min():.3f}")
    print(f"Max confidence:           {ground_truth_df['match_confidence'].max():.3f}")
    
    # Attribute winner distribution
    print("\n" + "=" * 60)
    print("BEST ATTRIBUTE SOURCE DISTRIBUTION:")
    print("=" * 60)
    
    source_labels = {0: 'non_base', 1: 'base', 2: 'tie'}
    
    print("\nNames:")
    name_counts = ground_truth_df['best_name_source'].value_counts()
    for source, count in name_counts.items():
        pct = (count / total_records) * 100
        source_label = source_labels.get(source, str(source))
        print(f"  {source_label:10s}: {count:4d} ({pct:5.1f}%)")
    
    print("\nCategories:")
    cat_counts = ground_truth_df['best_categories_source'].value_counts()
    for source, count in cat_counts.items():
        pct = (count / total_records) * 100
        source_label = source_labels.get(source, str(source))
        print(f"  {source_label:10s}: {count:4d} ({pct:5.1f}%)")
    
    print("\nAddresses:")
    addr_counts = ground_truth_df['best_address_source'].value_counts()
    for source, count in addr_counts.items():
        pct = (count / total_records) * 100
        source_label = source_labels.get(source, str(source))
        print(f"  {source_label:10s}: {count:4d} ({pct:5.1f}%)")
    
else:
    print("No ground truth results to analyze!")

# Export
ground_truth_df.to_csv('../output_data/entire_labeled_ground_truth.csv', index=False)

=== GROUND TRUTH COVERAGE ANALYSIS ===
Total matched pairs: 1639

NON-NULL COVERAGE ACROSS CORE ATTRIBUTES:

NAMES:
  Sample names:      1630 / 1639 ( 99.5%)
  Base names:        1628 / 1639 ( 99.3%)
  Yelp names:        1639 / 1639 (100.0%)

ADDRESSES:
  Sample addresses:  1637 / 1639 ( 99.9%)
  Base addresses:    1639 / 1639 (100.0%)
  Yelp addresses:    1639 / 1639 (100.0%)

CATEGORIES:
  Sample categories: 1626 / 1639 ( 99.2%)
  Base categories:   1591 / 1639 ( 97.1%)
  Yelp categories:   1639 / 1639 (100.0%)

MATCH QUALITY STATISTICS:
Average match confidence: 0.510
Match confidence std:     0.090
Min confidence:           0.400
Max confidence:           1.000

BEST ATTRIBUTE SOURCE DISTRIBUTION:

Names:
  tie       :  913 ( 55.7%)
  non_base  :  388 ( 23.7%)
  base      :  338 ( 20.6%)

Categories:
  tie       : 1554 ( 94.8%)
  non_base  :   52 (  3.2%)
  base      :   33 (  2.0%)

Addresses:
  non_base  :  596 ( 36.4%)
  tie       :  594 ( 36.2%)
  base      :  449 ( 27.4%)
